In [2]:
import pandas as pd
import os
import fnmatch
import numpy as np
from library import start
from library import clean_tea

In [3]:
start.data_path

'/Users/kylieleblancKylie/domino/dofis/data/'

In [4]:
year = 'yr1718'

# Certifications

In [81]:
folder = 'certification_' + year + '/'
teacher_datapath = os.path.join(start.data_path, 'tea', 'teachers', folder)

pattern = "CERTIFICATION*.csv"
if year == 'yr1213' or year == 'yr1314':
    pattern = "CERTIFICATION*.TXT"
cert_files = []
for entry in os.listdir(teacher_datapath):
    if fnmatch.fnmatch(entry, pattern):
        cert_files.append(entry)
cert_files.sort()
dirs_cert = [teacher_datapath + file for file in cert_files]
df_list = [pd.read_csv(file, sep=",", encoding="ISO-8859-1", dtype=object) for file in dirs_cert]
certification = pd.concat(df_list)
certification.head()

,PERSONID_SCRAM,DISTRICT,REGION,CERTIFICATE_ID_NUMBER,CERTIFICATE_LIFE,CREDENTIAL_TYPE,CERTIFICATE_PREPARATION_ROUTE,CERTIFICATE_EFFECTIVE_DATE,CERTIFICATE_EXPIRATION_DATE,SUBJECT_AREA,SUBJECT,FULLER_CERTIFICATE_DESCRIPTION,NUMBER_OF_RENEWAL_CYCLES,CREDENTIALED_GRADES,CERTIFICATION_LEVEL,ROLE_CREDENTIALED_FOR,POPULATION_CREDENTIALED_FOR,PREPARATION_PROGRAM,EMAIL_ADDRESS,MAILING_ADDRESS,CITY,ST,ZIP_CD,PHONE
0,*30*5P341,031901,01,2021478,LIFE,Paraprofessional,Paraprofessional Program,06OCT1994:00:00:00,NaN,Other,Not Applicable,Educational Aide I,NaN,NaN,Not Applicable,Educational Aide,Not Applicable,BROWNSVILLE ISD,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966
1,*30*5P341,031901,01,2502767,SYR,Emergency Non-Certified,Permit Program,09JAN2002:00:00:00,31AUG2002:00:00:00,Special Education,Generic Special Education,Generic Special Education/All Grades,NaN,Grades PK-12,Special Education,Teacher,Special Education,BROWNSVILLE ISD,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966
2,*30*5P341,031901,01,2595245,1YR,Probationary,Alternative Program,21AUG2002:00:00:00,21AUG2003:00:00:00,Special Education,Generic Special Education,Generic Special Education,NaN,Grades PK-12,Special Education,Teacher,Special Education,REG I EDUCATION SERVICE CENTER,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966
3,*30*5P341,031901,01,2744684,1YR,Probationary Extension,Alternative Program,31AUG2003:00:00:00,31AUG2004:00:00:00,Special Education,Generic Special Education,Generic Special Education,NaN,Grades PK-12,Special Education,Teacher,Special Education,REG I EDUCATION SERVICE CENTER,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966
4,*30*5P341,031901,01,2922393,5YR,Standard,Alternative Program,02APR2005:00:00:00,31OCT2010:00:00:00,Special Education,Generic Special Education,Special Education,1,Grades EC-12,Special Education,Teacher,Regular Students,REG I EDUCATION SERVICE CENTER,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966


In [82]:
vars_to_keep = {'PERSONID_SCRAM': 'teacher_id', 'DISTRICT': 'district',
                'CREDENTIAL_TYPE': 'cert_type', 'CERTIFICATE_PREPARATION_ROUTE': 'cert_route',
                'CERTIFICATION_LEVEL': 'cert_level', 'CREDENTIALED_GRADES': 'cert_grades',
                'SUBJECT_AREA': 'cert_area', 'SUBJECT': 'cert_subject'}
certification = clean_tea.filter_and_rename_cols(certification, vars_to_keep)
certification.head()

,teacher_id,district,cert_type,cert_route,cert_level,cert_grades,cert_area,cert_subject
0,*30*5P341,031901,Paraprofessional,Paraprofessional Program,Not Applicable,NaN,Other,Not Applicable
1,*30*5P341,031901,Emergency Non-Certified,Permit Program,Special Education,Grades PK-12,Special Education,Generic Special Education
2,*30*5P341,031901,Probationary,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education
3,*30*5P341,031901,Probationary Extension,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education
4,*30*5P341,031901,Standard,Alternative Program,Special Education,Grades EC-12,Special Education,Generic Special Education


In [92]:
certification[certification.cert_area == "Vocational Education"]

,teacher_id,district,cert_type,cert_route,cert_level,cert_grades,cert_area,cert_subject,cert_level_area
20,*3034L040,031901,Standard,Certification by Exam,Secondary,Grades 6-12,Vocational Education,Technology Education,Secondary - Vocational Education
21,*3034L040,031901,Standard,Certification by Exam,Secondary,Grades 6-12,Vocational Education,Business Education,Secondary - Vocational Education
23,*3034L040,031901,Standard,Certification by Exam,Secondary,Grades 6-12,Vocational Education,Technology Education,Secondary - Vocational Education
24,*3034L040,031901,Standard,Certification by Exam,Secondary,Grades 6-12,Vocational Education,Business Education,Secondary - Vocational Education
43,*31271Q42,031901,Probationary,Alternative Program,All Level,Grades EC-12,Vocational Education,Technology Applications,All Level - Vocational Education
44,*31271Q42,031901,Probationary Extension,Alternative Program,All Level,Grades EC-12,Vocational Education,Technology Applications,All Level - Vocational Education
45,*31271Q42,031901,Standard,Alternative Program,All Level,Grades EC-12,Vocational Education,Technology Applications,All Level - Vocational Education
46,*31271Q42,031901,Standard,Alternative Program,All Level,Grades EC-12,Vocational Education,Technology Applications,All Level - Vocational Education
189,*3347V342,031901,Probationary,Alternative Program,All Level,Grades EC-12,Vocational Education,Technology Applications,All Level - Vocational Education
190,*3347V342,031901,Standard,Alternative Program,All Level,Grades EC-12,Vocational Education,Technology Applications,All Level - Vocational Education


In [91]:
certification['cert_level_area'] = certification.cert_level + " - " +  certification.cert_area
pd.set_option('display.max_rows', 200)
certification.cert_level_area.value_counts()

Elementary - General Elementary (Self-Contained)     545387
Supplemental - Bilingual Education                   223931
Not Applicable - Other                               212637
Professional Service - Professional                  186732
Elementary - Bilingual Education                     115940
Special Education - Special Education                108771
Secondary - English Language Arts                     97408
Elementary - English Language Arts                    91965
All Level - Health and Physical Education             81101
Secondary - Social Studies                            79265
Secondary - Science                                   62961
Secondary - Mathematics                               47804
Elementary - Mathematics                              47766
All Level - Fine Arts                                 47558
Secondary - Vocational Education                      45498
All Level - Special Education                         40820
Endorsement - Bilingual Education       

In [84]:
certification.cert_area.value_counts()

General Elementary (Self-Contained)    566347
Bilingual Education                    387069
Other                                  224648
English Language Arts                  200578
Special Education                      194875
Professional                           190509
Health and Physical Education          123873
Social Studies                         108163
Mathematics                             95574
Science                                 86904
Vocational Education                    71807
Fine Arts                               65915
Foreign Language                        34798
Computer Science                         5043
Name: cert_area, dtype: int64

In [17]:
cert_types = {'Emergency Non-Certified': False, 'Emergency Certified': False, 
              'Emergency': False, 'Emergency Teaching': False,
              'Temporary Exemption': False, 'Temporary Teaching Certificate': False,
              'Unknown Permit': False, 'Unknown': False,
              'Special Assignment': False,
              'Paraprofessional': False, 'Standard Paraprofessional': False,
              'Standard': True, 'Provisional': True,
              'Probationary': True, 'Probationary Extension': True, 'Probationary Second Extension': True,
              'Non-renewable': True, 'One Year': True,
              'Visiting International Teacher': True,
              'Professional': True, 'Standard Professional':True}
certification['certification'] = certification['cert_type'].map(cert_types)
certification['vocational'] = np.where((certification['cert_type'] == "Vocational"), True, False)
certification.head()

,teacher_id,district,cert_type,cert_route,cert_level,cert_grades,cert_area,cert_subject,certification,vocational
0,*30*5P341,031901,Paraprofessional,Paraprofessional Program,Not Applicable,NaN,Other,Not Applicable,False,False
1,*30*5P341,031901,Emergency Non-Certified,Permit Program,Special Education,Grades PK-12,Special Education,Generic Special Education,False,False
2,*30*5P341,031901,Probationary,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False
3,*30*5P341,031901,Probationary Extension,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False
4,*30*5P341,031901,Standard,Alternative Program,Special Education,Grades EC-12,Special Education,Generic Special Education,True,False


In [18]:
grades_elem = {'Grades EC-4': True, 'Grades EC-6': True,'Grades EC-12': True,
               'Grades PK-KG': True, 'Grades PK-3': True, 'Grades PK-5': True,'Grades PK-12': True,
               'Grades 1-8': True,'Grades 1-6': True,
               'Grades 4-8': True,
               'Grades 6-8': False,'Grades 6-10': False, 'Grades 6-12': False,
               'Grades 7-12': False,
               'Grades 8-12': False}
grades_middle = {'Grades EC-4': False, 'Grades EC-6': True,'Grades EC-12': True,
               'Grades PK-KG': False, 'Grades PK-3': False, 'Grades PK-5': False,'Grades PK-12': True,
               'Grades 1-8': True,'Grades 1-6': True,
               'Grades 4-8': True,
               'Grades 6-8': True,'Grades 6-10': True, 'Grades 6-12': True,
               'Grades 7-12': True,
               'Grades 8-12': True}
grades_high = {'Grades EC-4': False, 'Grades EC-6': False,'Grades EC-12': True,
               'Grades PK-KG': False, 'Grades PK-3': False, 'Grades PK-5': False,'Grades PK-12': True,
               'Grades 1-8': False,'Grades 1-6': False,
               'Grades 4-8': False,
               'Grades 6-8': False,'Grades 6-10': True, 'Grades 6-12': True,
               'Grades 7-12': True,
               'Grades 8-12': True}
certification['cert_elem'] = certification['cert_grades'].map(grades_elem)
certification['cert_elem'] = certification.cert_elem.astype(bool)
certification['cert_elem'] = np.where((certification.certification == False), False, certification.cert_elem) #TODO code review

certification['cert_middle'] = certification['cert_grades'].map(grades_middle)
certification['cert_middle'] = certification.cert_middle.astype(bool)
certification['cert_middle'] = np.where((certification.certification == False), False, certification.cert_middle)

certification['cert_high'] = certification['cert_grades'].map(grades_high)
certification['cert_high'] = certification.cert_high.astype(bool)
certification['cert_high'] = np.where((certification.certification == False), False, certification.cert_high)

certification.head()

,teacher_id,district,cert_type,cert_route,cert_level,cert_grades,cert_area,cert_subject,certification,vocational,cert_elem,cert_middle,cert_high
0,*30*5P341,031901,Paraprofessional,Paraprofessional Program,Not Applicable,NaN,Other,Not Applicable,False,False,False,False,False
1,*30*5P341,031901,Emergency Non-Certified,Permit Program,Special Education,Grades PK-12,Special Education,Generic Special Education,False,False,False,False,False
2,*30*5P341,031901,Probationary,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False,True,True,True
3,*30*5P341,031901,Probationary Extension,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False,True,True,True
4,*30*5P341,031901,Standard,Alternative Program,Special Education,Grades EC-12,Special Education,Generic Special Education,True,False,True,True,True


In [80]:
certification['cert_area_elem'] = np.where(certification['cert_area'] == "General Elementary (Self-Contained)", True, False)
certification['cert_area_ela'] = np.where(certification['cert_area'] == "English Language Arts", True, False)
certification['cert_area_math'] = np.where(certification['cert_area'] == "Mathematics", True, False)
certification['cert_area_sci'] = np.where(certification['cert_area'] == "Science", True, False)
certification.head(150)

,teacher_id,district,cert_level,cert_area,certification,vocational,cert_elem,cert_middle,cert_high,cert_area_elem,cert_area_ela,cert_area_math,cert_area_sci
0,*30*5P341,031901,Not Applicable,Other,False,False,False,False,False,False,False,False,False
1,*30*5P341,031901,Special Education,Special Education,False,False,False,False,False,False,False,False,False
2,*30*5P341,031901,Special Education,Special Education,True,False,True,True,True,False,False,False,False
3,*30*5P341,031901,Special Education,Special Education,True,False,True,True,True,False,False,False,False
4,*30*5P341,031901,Special Education,Special Education,True,False,True,True,True,False,False,False,False
5,*30*5P341,031901,Special Education,Special Education,True,False,True,True,True,False,False,False,False
6,*30*5P341,031901,Special Education,Special Education,True,False,True,True,True,False,False,False,False
7,*30*7QV40,031901,Not Applicable,Other,False,False,False,False,False,False,False,False,False
8,*30*7QV40,031901,Not Applicable,Other,False,False,False,False,False,False,False,False,False
9,*3005Q041,031901,Not Applicable,Other,False,False,False,False,False,False,False,False,False


In [20]:
certification = certification[['teacher_id', 'district', 'cert_level', 'cert_area', 'certification', 'vocational', 
                                 'cert_elem', 'cert_middle', 'cert_high',
                                 'cert_area_elem', 'cert_area_ela', 'cert_area_math', 'cert_area_sci']]
certification.sample(n=10)

,teacher_id,district,cert_level,cert_area,certification,vocational,cert_elem,cert_middle,cert_high,cert_area_elem,cert_area_ela,cert_area_math,cert_area_sci
2423,13253*443,043901,Elementary,English Language Arts,True,False,True,True,False,False,True,False,False
30810,V360VQ*46,181907,Professional Service,Professional,True,False,True,True,True,False,False,False,False
19075,P34V2Y334,139909,Elementary,Bilingual Education,True,False,True,True,False,False,False,False,False
24116,*3Q3Q0330,094901,Secondary,English Language Arts,True,False,False,True,True,False,True,False,False
7568,Q31V01447,020901,Elementary,English Language Arts,True,False,True,True,False,False,True,False,False
248033,Q335QP246,220914,Elementary,General Elementary (Self-Contained),True,False,True,True,False,True,False,False,False
217619,V32*71041,220907,Endorsement,General Elementary (Self-Contained),True,False,True,False,False,True,False,False,False
68567,23Q111V47,072801,Not Applicable,Other,False,False,False,False,False,False,False,False,False
496295,03815FP46,101925,Secondary,Science,True,False,False,True,True,False,False,False,True
54561,131*5D*40,061911,Professional Service,Professional,False,False,False,False,False,False,False,False,False


In [28]:
certification[(certification.district == '001902') & (certification.cert_level == "Secondary")]

,teacher_id,district,cert_level,cert_area,certification,vocational,cert_elem,cert_middle,cert_high,cert_area_elem,cert_area_ela,cert_area_math,cert_area_sci
15,*3513P038,001902,Secondary,Science,True,False,False,True,True,False,False,False,True
20,*3513P038,001902,Secondary,Science,True,False,False,True,True,False,False,False,True
25,*3647LV46,001902,Secondary,Vocational Education,True,False,False,True,True,False,False,False,False
26,*3647LV46,001902,Secondary,Vocational Education,True,False,False,True,True,False,False,False,False
27,030Q3Y*42,001902,Secondary,Social Studies,True,False,False,True,True,False,False,False,False
34,03127*449,001902,Secondary,Social Studies,True,False,False,True,True,False,False,False,False
35,03127*449,001902,Secondary,Social Studies,True,False,False,True,True,False,False,False,False
37,03127*449,001902,Secondary,Social Studies,True,False,False,True,True,False,False,False,False
39,03127*449,001902,Secondary,Social Studies,True,False,False,True,True,False,False,False,False
40,03127*449,001902,Secondary,Social Studies,True,False,False,True,True,False,False,False,False


In [21]:
certification.dtypes

teacher_id        object
district          object
cert_level        object
cert_area         object
certification     object
vocational          bool
cert_elem           bool
cert_middle       object
cert_high         object
cert_area_elem      bool
cert_area_ela       bool
cert_area_math      bool
cert_area_sci       bool
dtype: object

## Collapse to teacher level

In [74]:
teacher_cert = certification[['teacher_id', 'district', 'certification', 'vocational', 
                                 'cert_elem', 'cert_middle', 'cert_high',
                                 'cert_area_elem', 'cert_area_ela', 'cert_area_math', 'cert_area_sci']]

In [75]:
teacher_cert = teacher_cert.groupby(['teacher_id']).max()

AttributeError: 'DataFrame' object has no attribute 'teacher_id'

In [78]:
teacher_cert[teacher_cert.district == '001902']

,district,certification,vocational,cert_elem,cert_middle,cert_high,cert_area_elem,cert_area_ela,cert_area_math,cert_area_sci
teacher_id,,,,,,,,,,
*3242P*44,001902,True,False,True,True,True,True,False,False,False
*3353*046,001902,False,False,False,False,False,False,False,False,False
*3513P038,001902,True,False,True,True,True,True,False,True,True
*3623LV43,001902,False,False,False,False,False,False,False,False,False
*3647LV46,001902,True,False,False,True,True,False,False,False,False
030Q3Y*42,001902,True,False,True,True,True,True,False,False,False
03127*449,001902,True,False,True,True,True,False,False,False,False
032V2V137,001902,True,False,True,True,False,True,False,False,False
033*QP545,001902,True,False,True,True,False,True,True,False,False


In [53]:
teacher_cert[teacher_cert.teacher_id == 'Q3Q2QF*48']

,teacher_id,certification,vocational,cert_elem,cert_middle,cert_high,cert_area_elem,cert_area_ela,cert_area_math,cert_area_sci
234,Q3Q2QF*48,True,False,True,True,False,True,False,False,False
235,Q3Q2QF*48,True,False,True,True,True,False,False,False,False
236,Q3Q2QF*48,True,False,True,True,False,True,False,False,False
237,Q3Q2QF*48,True,False,True,True,False,True,False,False,False
238,Q3Q2QF*48,True,False,True,True,True,False,False,False,False


## Sort and to csv

In [11]:
teacher_cert.sort_values(by = ['teacher_id'], axis = 0)
filename = 'teacher_cert' + year + '.csv'
certification.to_csv(os.path.join(start.data_path, 'tea', 'teachers', filename))

# Teacher Assignments

In [40]:
pattern = "TEACHER_MASTER*.TXT"
teacher_files = []
for entry in os.listdir(teacher_datapath):
    if fnmatch.fnmatch(entry, pattern):
        teacher_files.append(entry)
teacher_files.sort()
dirs_teachers = [teacher_datapath + file for file in teacher_files]
df_list = [pd.read_csv(file, sep=",", encoding="ISO-8859-1", dtype=object) for file in dirs_teachers]
teachers = pd.concat(df_list)
teachers = teachers[teachers['ROLE NAME'] == 'TEACHER']

In [43]:
teachers[(teachers['DISTRICT NUMBER'] == '001902') & (teachers['CAMPUS NAME'] == 'CAYUGA H S')]

,REGION NUMBER,COUNTY NUMBER,COUNTY NAME,DISTRICT NUMBER,DISTRICT NAME,DISTRICT TYPE CODE,DISTRICT TYPE NAME,DISTRICT CHARTER TYPE CODE,DISTRICT CHARTER TYPE NAME,DISTRICT CATEGORY CODE,DISTRICT CATEGORY NAME,DISTRICT PHONE,DISTRICT FAX,DISTRICT ADDRESS,DISTRICT CITY,DISTRICT STATE,DISTRICT ZIP,CAMPUS NAME,CAMPUS LOW GRADE,CAMPUS HIGH GRADE,CAMPUS TYPE CODE,CAMPUS TYPE NAME,CAMPUS CHARTER TYPE CODE,CAMPUS CHARTER TYPE NAME,CAMPUS GRADE GROUP CODE,CAMPUS GRADE GROUP NAME,CAMPUS ADDRESS,CAMPUS CITY,CAMPUS STATE,CAMPUS ZIP,CAMPUS PHONE,CAMPUS EMAIL,SCRAMBLED UNIQUE ID,FIRST NAME,MIDDLE NAME,LAST NAME,SEX CODE,SEX NAME,ETHNICITY CODE,ETHNICITY NAME,EXPERIENCE,DEGREE CODE,DEGREE NAME,FTE,BASE PAY,OTHER PAY,TOTAL PAY,ROLE CODE,ROLE NAME,ROLE FULL TIME EQUIVALENT,ROLE BASE PAY,SUBJECT AREA CODE 1,SUBJECT AREA NAME 1,SUBJECT AREA CODE 2,SUBJECT AREA NAME 2,SUBJECT AREA CODE 3,SUBJECT AREA NAME 3,SUBJECT AREA CODE 4,SUBJECT AREA NAME 4,SUBJECT AREA CODE 5,SUBJECT AREA NAME 5,SUBJECT AREA CODE 6,SUBJECT AREA NAME 6,SUBJECT AREA CODE 7,SUBJECT AREA NAME 7,SUBJECT AREA CODE 8,SUBJECT AREA NAME 8,SUBJECT AREA CODE 9,SUBJECT AREA NAME 9,SUBJECT AREA CODE 10,SUBJECT AREA NAME 10,PAY TYPE CODE 1,PAY TYPE NAME 1,PAY TYPE CODE 2,PAY TYPE NAME 2,PAY TYPE CODE 3,PAY TYPE NAME 3,PAY TYPE CODE 4,PAY TYPE NAME 4,PAY TYPE CODE 5,PAY TYPE NAME 5,EMAIL,CAMPUS NUMBER
1,07,001,ANDERSON COUNTY,001902,CAYUGA ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,H,RURAL,(903)928-2102,(903)928-2646,P O BOX 427,CAYUGA,TX,75832-0427,CAYUGA H S,09,12,1,INSTRUCTIONAL CAMPUS,00,NOT A CHARTER SCHOOL,4,HIGH SCHOOL,P O BOX 427,CAYUGA,TX,75832-0427,(903)928-2294,admin1@cayugaisd.com,030Q3Y*42,STEPHEN,L,DRINKARD,M,MALE,W,White,20,1,BACHELOR'S,0.9733,47875,7500,55375,087,TEACHER,0.2164,10648,06,PHYSICAL ED. & HEALTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02,UIL ATHLETICS,80,BASE SALARY,NaN,NaN,NaN,NaN,NaN,NaN,admin1@cayugaisd.com,001902001
2,07,001,ANDERSON COUNTY,001902,CAYUGA ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,H,RURAL,(903)928-2102,(903)928-2646,P O BOX 427,CAYUGA,TX,75832-0427,CAYUGA H S,09,12,1,INSTRUCTIONAL CAMPUS,00,NOT A CHARTER SCHOOL,4,HIGH SCHOOL,P O BOX 427,CAYUGA,TX,75832-0427,(903)928-2294,admin1@cayugaisd.com,03127*449,EDDIE,L,ERVIN,M,MALE,W,White,12,1,BACHELOR'S,0.9733,41785,10500,52285,087,TEACHER,0.6195,26586,05,SOCIAL STUDIES,06,PHYSICAL ED. & HEALTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02,UIL ATHLETICS,80,BASE SALARY,NaN,NaN,NaN,NaN,NaN,NaN,admin1@cayugaisd.com,001902001
6,07,001,ANDERSON COUNTY,001902,CAYUGA ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,H,RURAL,(903)928-2102,(903)928-2646,P O BOX 427,CAYUGA,TX,75832-0427,CAYUGA H S,09,12,1,INSTRUCTIONAL CAMPUS,00,NOT A CHARTER SCHOOL,4,HIGH SCHOOL,P O BOX 427,CAYUGA,TX,75832-0427,(903)928-2294,admin1@cayugaisd.com,03460Y544,JEFFREY,RYAN,QUATTLEBAUM,M,MALE,W,White,11,1,BACHELOR'S,0.9733,40815,0,40815,087,TEACHER,0.9732,40824,01,SELF-CONTAINED,05,SOCIAL STUDIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80,BASE SALARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,admin1@cayugaisd.com,001902001
10,07,001,ANDERSON COUNTY,001902,CAYUGA ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,H,RURAL,(903)928-2102,(903)928-2646,P O BOX 427,CAYUGA,TX,75832-0427,CAYUGA H S,09,12,1,INSTRUCTIONAL CAMPUS,00,NOT A CHARTER SCHOOL,4,HIGH SCHOOL,P O BOX 427,CAYUGA,TX,75832-0427,(903)928-2294,admin1@cayugaisd.com,03762P246,ELIZABETH,ASHLEY,BRYANT,F,FEMALE,W,White,15,1,BACHELOR'S,0.9733,44395,0,44395,087,TEACHER,0.0565,2579,99,OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80,BASE SALARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,admin1@cayugaisd.com,001902001
11,07,001,ANDERSON COUNTY,001902,CAYUGA ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,H,RURAL,(903)928-2102,(903)928-2646,P O BOX 427,CAYUGA,TX,75832-0427,CAYUGA H S,09,12,1,INSTRUCTIONAL CAMPUS,00,

In [44]:
pd.set_option('display.max_columns', 100)
teachers

,REGION NUMBER,COUNTY NUMBER,COUNTY NAME,DISTRICT NUMBER,DISTRICT NAME,DISTRICT TYPE CODE,DISTRICT TYPE NAME,DISTRICT CHARTER TYPE CODE,DISTRICT CHARTER TYPE NAME,DISTRICT CATEGORY CODE,DISTRICT CATEGORY NAME,DISTRICT PHONE,DISTRICT FAX,DISTRICT ADDRESS,DISTRICT CITY,DISTRICT STATE,DISTRICT ZIP,CAMPUS NAME,CAMPUS LOW GRADE,CAMPUS HIGH GRADE,CAMPUS TYPE CODE,CAMPUS TYPE NAME,CAMPUS CHARTER TYPE CODE,CAMPUS CHARTER TYPE NAME,CAMPUS GRADE GROUP CODE,CAMPUS GRADE GROUP NAME,CAMPUS ADDRESS,CAMPUS CITY,CAMPUS STATE,CAMPUS ZIP,CAMPUS PHONE,CAMPUS EMAIL,SCRAMBLED UNIQUE ID,FIRST NAME,MIDDLE NAME,LAST NAME,SEX CODE,SEX NAME,ETHNICITY CODE,ETHNICITY NAME,EXPERIENCE,DEGREE CODE,DEGREE NAME,FTE,BASE PAY,OTHER PAY,TOTAL PAY,ROLE CODE,ROLE NAME,ROLE FULL TIME EQUIVALENT,ROLE BASE PAY,SUBJECT AREA CODE 1,SUBJECT AREA NAME 1,SUBJECT AREA CODE 2,SUBJECT AREA NAME 2,SUBJECT AREA CODE 3,SUBJECT AREA NAME 3,SUBJECT AREA CODE 4,SUBJECT AREA NAME 4,SUBJECT AREA CODE 5,SUBJECT AREA NAME 5,SUBJECT AREA CODE 6,SUBJECT AREA NAME 6,SUBJECT AREA CODE 7,SUBJECT AREA NAME 7,SUBJECT AREA CODE 8,SUBJECT AREA NAME 8,SUBJECT AREA CODE 9,SUBJECT AREA NAME 9,SUBJECT AREA CODE 10,SUBJECT AREA NAME 10,PAY TYPE CODE 1,PAY TYPE NAME 1,PAY TYPE CODE 2,PAY TYPE NAME 2,PAY TYPE CODE 3,PAY TYPE NAME 3,PAY TYPE CODE 4,PAY TYPE NAME 4,PAY TYPE CODE 5,PAY TYPE NAME 5,EMAIL,CAMPUS NUMBER
0,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,C,OTHER CENTRAL CITY,(956)548-8000,(956)574-6497,1900 PRICE RD,BROWNSVILLE,TX,78521-2417,SHARP EL,EE,05,1,INSTRUCTIONAL CAMPUS,00,NOT A CHARTER SCHOOL,1,ELEMENTARY,1439 PALM BLVD,BROWNSVILLE,TX,78520-7292,(956)982-2930,igsegura@bisd.us,*30*5P341,NORMA,ALICIA,GANDARA,F,FEMALE,H,Hispanic/Latino,17,1,BACHELOR'S,1.0000,53130,3500,56630,087,TEACHER,1.0002,53141,01,SELF-CONTAINED,02,ENGLISH LANGUAGE ARTS,03,MATHEMATICS,04,SCIENCE,05,SOCIAL STUDIES,06,PHYSICAL ED. & HEALTH,09,TECHNOLOGY APPLICATIONS,99,OTHER,NaN,NaN,NaN,NaN,79,OTHER SUPPLEMENTAL DUTIES,80,BASE SALARY,NaN,NaN,NaN,NaN,NaN,NaN,drezendejas@bisd.us,031901111
1,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,C,OTHER CENTRAL CITY,(956)548-8000,(956)574-6497,1900 PRICE RD,BROWNSVILLE,TX,78521-2417,OLIVEIRA MIDDLE,06,08,1,INSTRUCTIONAL CAMPUS,00,NOT A CHARTER SCHOOL,2,MIDDLE SCHOOL,444 LAND O' LAKES,BROWNSVILLE,TX,78521-4296,(956)548-8530,cscastro@bisd.us,*3034L040,JOSE,NaN,REBOLLO,M,MALE,H,Hispanic/Latino,16,2,MASTER'S,1.0000,55402,0,55402,087,TEACHER,1.0000,55400,11,CAREER & TECHNOLOGY EDUCATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80,BASE SALARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drezendejas@bisd.us,031901045
2,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,C,OTHER CENTRAL CITY,(956)548-8000,(956)574-6497,1900 PRICE RD,BROWNSVILLE,TX,78521-2417,GARDEN PARK EL,EE,05,1,INSTRUCTIONAL CAMPUS,00,NOT A CHARTER SCHOOL,1,ELEMENTARY,855 MILITARY HWY,BROWNSVILLE,TX,78520-4661,(956)982-2630,vcaballero@bisd.us,*3040V545,ANNA,M,ORTEGA,F,FEMALE,H,Hispanic/Latino,33,2,MASTER'S,1.0000,67357,750,68107,087,TEACHER,0.9999,67349,02,ENGLISH LANGUAGE ARTS,03,MATHEMATICS,04,SCIENCE,05,SOCIAL STUDIES,06,PHYSICAL ED. & HEALTH,07,FOREIGN LANGUAGE,09,TECHNOLOGY APPLICATIONS,99,OTHER,NaN,NaN,NaN,NaN,79,OTHER SUPPLEMENTAL DUTIES,80,BASE SALARY,NaN,NaN,NaN,NaN,NaN,NaN,drezendejas@bisd.us,031901106
3,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,C,OTHER CENTRAL CITY,(956)548-8000,(956)574-6497,1900 PRICE RD,BROWNSVILLE,TX,78521-2417,MORNINGSIDE EL,EE,05,1,INSTRUCTIONAL CAMPUS,00,NOT A CHARTER SCHOOL,1,ELEMENTARY,1025 MORNINGSIDE RD,BROWNSVILLE,TX,78521-7635,(956)982-2760,jhmartinez@bisd.us,*3106V240,MONICA,NaN,GUTIERREZ,F,FEMALE,H,Hispanic/Latino,0,0,NO BACHELOR'S DEGREE OR HIGHER,1.0000,16172,0,16172,087,TEACHER,0.0496,802,99,OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [45]:
vars_to_keep = {'SCRAMBLED UNIQUE ID': 'teacher_id', 'DISTRICT NUMBER': 'district', 'DISTRICT NAME': 'distname',
                        'CAMPUS NUMBER': 'campus', 'CAMPUS NAME': 'campname',
                        'CAMPUS GRADE GROUP NAME': 'camp_grade_group',
                        'FTE': 'fte', 'ROLE FULL TIME EQUIVALENT': 'fte_teacher',
                        'SUBJECT AREA NAME 1': 'sub_area1', 'SUBJECT AREA NAME 2': 'sub_area2',
                        'SUBJECT AREA NAME 3': 'sub_area3',
                        'SUBJECT AREA NAME 4': 'sub_area4', 'SUBJECT AREA NAME 5': 'sub_area5'}
teachers = clean_tea.filter_and_rename_cols(teachers, vars_to_keep)
teachers.head()

,teacher_id,district,distname,campus,campname,camp_grade_group,fte,fte_teacher,sub_area1,sub_area2,sub_area3,sub_area4,sub_area5
0,*30*5P341,031901,BROWNSVILLE ISD,031901111,SHARP EL,ELEMENTARY,1.0000,1.0002,SELF-CONTAINED,ENGLISH LANGUAGE ARTS,MATHEMATICS,SCIENCE,SOCIAL STUDIES
1,*3034L040,031901,BROWNSVILLE ISD,031901045,OLIVEIRA MIDDLE,MIDDLE SCHOOL,1.0000,1.0000,CAREER & TECHNOLOGY EDUCATION,NaN,NaN,NaN,NaN
2,*3040V545,031901,BROWNSVILLE ISD,031901106,GARDEN PARK EL,ELEMENTARY,1.0000,0.9999,ENGLISH LANGUAGE ARTS,MATHEMATICS,SCIENCE,SOCIAL STUDIES,PHYSICAL ED. & HEALTH
3,*3106V240,031901,BROWNSVILLE ISD,031901129,MORNINGSIDE EL,ELEMENTARY,1.0000,0.0496,OTHER,NaN,NaN,NaN,NaN
4,*31271Q42,031901,BROWNSVILLE ISD,031901003,PACE EARLY COLLEGE H S,HIGH SCHOOL,1.0000,0.6426,MATHEMATICS,SOCIAL STUDIES,PHYSICAL ED. & HEALTH,FOREIGN LANGUAGE,NaN


In [32]:
teachers.sub_area1.value_counts()

ENGLISH LANGUAGE ARTS            111213
SELF-CONTAINED                    51742
MATHEMATICS                       42564
NOT APPLICABLE                    26686
SCIENCE                           24477
PHYSICAL ED. & HEALTH             22294
SOCIAL STUDIES                    21670
FINE ARTS                         19666
CAREER & TECHNOLOGY EDUCATION     14664
SPECIAL EDUCATION                 10626
OTHER                              7581
FOREIGN LANGUAGE                   6883
TECHNOLOGY APPLICATIONS            1947
Name: sub_area1, dtype: int64

In [33]:
teachers.camp_grade_group.value_counts()

ELEMENTARY                166416
HIGH SCHOOL               102949
MIDDLE SCHOOL              62879
ELEMENTARY / SECONDARY     16878
JUNIOR HIGH SCHOOL         11483
OTHER GRADE GROUP           1408
Name: camp_grade_group, dtype: int64

In [34]:
teachers['campus_elem'] = np.where((teachers.camp_grade_group == "ELEMENTARY") | (teachers.camp_grade_group == "ELEMENTARY / SECONDARY"), True, False)
teachers['campus_middle'] = np.where((teachers.camp_grade_group == "MIDDLE SCHOOL") | (teachers.camp_grade_group == "ELEMENTARY / SECONDARY"), True, False)
teachers['campus_high'] = np.where((teachers.camp_grade_group == "HIGH SCHOOL") | (teachers.camp_grade_group == "ELEMENTARY / SECONDARY"), True, False)

In [46]:
teachers['course_elem'] = np.where(((teachers.sub_area1 == "SELF-CONTAINED") |
                                     (teachers.sub_area2 == "SELF-CONTAINED") |
                                     (teachers.sub_area3 == "SELF-CONTAINED") |
                                     (teachers.sub_area4 == "SELF-CONTAINED") |
                                     (teachers.sub_area5 == "SELF-CONTAINED")), True, False)
teachers['course_math'] = np.where(((teachers.sub_area1 == "MATHEMATICS") |
                                     (teachers.sub_area2 == "MATHEMATICS") |
                                     (teachers.sub_area3 == "MATHEMATICS") |
                                     (teachers.sub_area4 == "MATHEMATICS") |
                                     (teachers.sub_area5 == "MATHEMATICS")), True, False)
teachers['course_science'] = np.where(((teachers.sub_area1 == "SCIENCE") |
                                     (teachers.sub_area2 == "SCIENCE") |
                                     (teachers.sub_area3 == "SCIENCE") |
                                     (teachers.sub_area4 == "SCIENCE") |
                                     (teachers.sub_area5 == "SCIENCE")), True, False)
teachers['course_cte'] = np.where(((teachers.sub_area1 == "CAREER & TECHNOLOGY EDUCATION") |
                                     (teachers.sub_area2 == "CAREER & TECHNOLOGY EDUCATION") |
                                     (teachers.sub_area3 == "CAREER & TECHNOLOGY EDUCATION") |
                                     (teachers.sub_area4 == "CAREER & TECHNOLOGY EDUCATION") |
                                     (teachers.sub_area5 == "CAREER & TECHNOLOGY EDUCATION")), True, False)

In [48]:
teachers[teachers.campus == '001902001']

,teacher_id,district,distname,campus,campname,camp_grade_group,fte,fte_teacher,sub_area1,sub_area2,sub_area3,sub_area4,sub_area5,course_elem,course_math,course_science,course_cte
1,030Q3Y*42,001902,CAYUGA ISD,001902001,CAYUGA H S,HIGH SCHOOL,0.9733,0.2164,PHYSICAL ED. & HEALTH,NaN,NaN,NaN,NaN,False,False,False,False
2,03127*449,001902,CAYUGA ISD,001902001,CAYUGA H S,HIGH SCHOOL,0.9733,0.6195,SOCIAL STUDIES,PHYSICAL ED. & HEALTH,NaN,NaN,NaN,False,False,False,False
6,03460Y544,001902,CAYUGA ISD,001902001,CAYUGA H S,HIGH SCHOOL,0.9733,0.9732,SELF-CONTAINED,SOCIAL STUDIES,NaN,NaN,NaN,True,False,False,False
10,03762P246,001902,CAYUGA ISD,001902001,CAYUGA H S,HIGH SCHOOL,0.9733,0.0565,OTHER,NaN,NaN,NaN,NaN,False,False,False,False
11,13002V245,001902,CAYUGA ISD,001902001,CAYUGA H S,HIGH SCHOOL,1.0000,0.6064,PHYSICAL ED. & HEALTH,NaN,NaN,NaN,NaN,False,False,False,False
14,2333VV330,001902,CAYUGA ISD,001902001,CAYUGA H S,HIGH SCHOOL,1.0000,0.4444,FINE ARTS,NaN,NaN,NaN,NaN,False,False,False,False
15,F35*2YV44,001902,CAYUGA ISD,001902001,CAYUGA H S,HIGH SCHOOL,0.9733,0.9737,ENGLISH LANGUAGE ARTS,NaN,NaN,NaN,NaN,False,False,False,False
16,F3Q000*38,001902,CAYUGA ISD,001902001,CAYUGA H S,HIGH SCHOOL,0.9733,0.6488,ENGLISH LANGUAGE ARTS,NaN,NaN,NaN,NaN,False,False,False,False
18,P3021F049,001902,CAYUGA ISD,001902001,CAYUGA H S,HIGH SCHOOL,0.9733,0.3892,PHYSICAL ED. & HEALTH,NaN,NaN,NaN,NaN,False,False,False,False
19,P35131043,001902,CAYUGA ISD,001902001,CAYUGA H S,HIGH SCHOOL,0.9733,0.7965,PHYSICAL ED. & HEALTH,CAREER & TECHNOLOGY EDUCATION,NaN,NaN,NaN,False,False,False,True


In [36]:
teachers = teachers[['teacher_id', 'district', 'campus', 'campus_elem', 'campus_middle', 'campus_high', 'course_elem', 'course_math', 'course_science', 'course_cte']]
teachers.head()

,teacher_id,district,campus,campus_elem,campus_middle,campus_high,course_elem,course_math,course_science,course_cte
0,*30*5P341,031901,031901111,True,False,False,True,True,True,False
1,*3034L040,031901,031901045,False,True,False,False,False,False,True
2,*3040V545,031901,031901106,True,False,False,False,True,True,False
3,*3106V240,031901,031901129,True,False,False,False,False,False,False
4,*31271Q42,031901,031901003,False,False,True,False,True,False,False


In [39]:
teachers[teachers.district == '001902']

,teacher_id,district,campus,campus_elem,campus_middle,campus_high,course_elem,course_math,course_science,course_cte
1,030Q3Y*42,001902,001902001,False,False,True,False,False,False,False
2,03127*449,001902,001902001,False,False,True,False,False,False,False
3,032V2V137,001902,001902103,True,False,False,False,True,True,False
4,033*QP545,001902,001902103,True,False,False,False,True,False,False
5,03407YV49,001902,001902103,True,False,False,False,True,True,False
6,03460Y544,001902,001902001,False,False,True,True,False,False,False
7,03554P144,001902,001902041,False,True,False,False,False,False,False
8,03602V*48,001902,001902041,False,True,False,False,False,False,False
9,03725P348,001902,001902103,True,False,False,False,False,False,False
10,03762P246,001902,001902001,False,False,True,False,False,False,False


In [51]:
teachers.sort_values(by = ['teacher_id'], axis = 0)
filename = 'teacher_course_' + year + '.csv'
teachers.to_csv(os.path.join(start.data_path, 'tea', 'teachers', filename))